# Import libraries

In [0]:
import tensorflow as tf
import numpy as np
import os

import keras
from keras.models import Sequential
from keras.layers import Input,Flatten, Conv2D, MaxPooling2D,UpSampling2D,add, concatenate
from keras.layers.convolutional import Convolution2D
from keras.layers.core import Dropout, Activation
from keras.layers.pooling import GlobalAveragePooling2D
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras import optimizers

Using TensorFlow backend.


# Preparing the dataset 

In [0]:
#######################################################
# Import training input
train_images = np.load('Npy_data/Train_image.npy')
train_labels = np.load('Npy_data/Train_den.npy')
print('training preparation is completed')

#######################################################
# Import validation input
test_images = np.load('Npy_data/Test_image.npy')
test_labels = np.load('Npy_data/Test_den.npy')
print('Tesing preparation is completed')

# Normalized data
TLs_max = train_labels.max()
TLs_min = train_labels.min()

train_images = (train_images - train_images.min()) / (train_images.max() - train_images.min())
train_labels = (train_labels - train_labels.min()) / (train_labels.max() - train_labels.min())


training preparation is completed
Val preparation is completed


# Hyperparameter

In [0]:
factor =3         # for control number of filter in the network
num_epoch = 50
batch_size = 1

# Slave network architecture

In [0]:
x_p = tf.placeholder(shape=[None,480,640,3],dtype=tf.float32)
y_p = tf.placeholder(shape=[None,480,640,1],dtype=tf.float32)
conv_11_p = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(x_p)
conv_12_p = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_11_p)
conv_13_p = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_12_p)
pool_1_p  = MaxPooling2D(pool_size=(2,2))(conv_13_p)

conv_21_p = Conv2D(16*factor, (3, 3), activation='relu', padding='same',trainable = True)(pool_1_p)
conv_22_p = Conv2D(16*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_21_p)
pool_2_p  = MaxPooling2D(pool_size=(2,2))(conv_22_p)

conv_31_p = Conv2D(32*factor, (3, 3), activation='relu', padding='same',trainable = True)(pool_2_p)
conv_32_p = Conv2D(32*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_31_p)
pool_3_p  = MaxPooling2D(pool_size=(2,2))(conv_32_p)

conv_41_p = Conv2D(64*factor, (3, 3), activation='relu', padding='same',trainable = True)(pool_3_p)
conv_42_p = Conv2D(64*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_41_p)



up_5_p   = UpSampling2D(size = (2,2))(conv_42_p)
# conv_51_p = Conv2D(32*factor, (3, 3), activation='relu', padding='same',trainable = True)(concatenate([up_5_p, conv_32_p], axis=-1))
conv_51_p = Conv2D(32*factor, (3, 3), activation='relu', padding='same',trainable = True)(up_5_p)
conv_52_p = Conv2D(32*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_51_p)
out_3                  = conv_52_p


up_6_p   = UpSampling2D(size = (2,2))(conv_52_p)
# conv_61_p = Conv2D(16*factor, (3, 3), activation='relu', padding='same',trainable = True)(concatenate([up_6_p, conv_22_p], axis=-1))
conv_61_p = Conv2D(16*factor, (3, 3), activation='relu', padding='same',trainable = True)(up_6_p)
conv_62_p = Conv2D(16*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_61_p)
out_2                  = conv_62_p


up_7_p  = UpSampling2D(size = (2,2))(conv_62_p)
# conv_71_p = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(concatenate([up_7_p, conv_13_p], axis=-1))
conv_71_p = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(up_7_p)
conv_72_p = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_71_p)
conv_73_p = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_72_p)
out_1                  = conv_73_p 

layer_final_p = Conv2D(1, (1, 1), activation='linear')(conv_73_p)

cost_p = tf.losses.mean_squared_error(y_p,layer_final_p)
auto_train_p = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost_p)


# For save the model weight in each layer in the slave network
saver_slave=tf.train.Saver()
save_dir_slave = 'Save_weighted/Slave_network/'
if not os.path.exists(save_dir_slave):
    os.makedirs(save_dir_slave)
save_path_slave = os.path.join(save_dir_slave, 'best_validation')





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# Restore weight section of slave network

In [0]:
sess1 = tf.Session()
sess1.run(tf.global_variables_initializer())

# Restore the weight
saver_slave.restore(sess1, "Save_weighted/Slave_network/best_validation")
print("Restore model successfully")

for current_batch_index in range(0,len(train_images),batch_size):
  current_batch = train_images[current_batch_index:current_batch_index+batch_size,:,:,:]
  current_label = train_labels[current_batch_index:current_batch_index+batch_size,:,:,:]
  sess1_results = sess1.run([cost_p,out_1,out_2,out_3,layer_final_p],
                                      feed_dict = {x_p:current_batch,
                                                        y_p:current_label,
                                                          })
  P_1 = sess1_results[1]
  P_2 = sess1_results[2]
  P_3 = sess1_results[3]

INFO:tensorflow:Restoring parameters from Save_weighted/Slave_network/best_validation
Restore model successfully


# Training the slave network

In [0]:
########### Training Slave network ####################
sess1 = tf.Session()
sess1.run(tf.global_variables_initializer())

train_mae =0.0
MSE_matrix = np.zeros([num_epoch,2])
Max_error = 99999

for iter in range(num_epoch):
    MSE =0
    len_train =0
    for current_batch_index in range(0,len(train_images),batch_size):
        current_batch = train_images[current_batch_index:current_batch_index+batch_size,:,:,:]
        current_label = train_labels[current_batch_index:current_batch_index+batch_size,:,:,:]

        sess1_results = sess1.run([cost_p,auto_train_p,out_1,out_2,out_3,layer_final_p],
                                      feed_dict = {x_p:current_batch,
                                                        y_p:current_label,
                                                          })
        P_1 = sess1_results[2]  ## Saved the pretrained feature map for the master network
        P_2 = sess1_results[3]	## Saved the pretrained feature map for the master network
        P_3 = sess1_results[4]	## Saved the pretrained feature map for the master network
        print(' Pretrained model Iter: ', iter, " Cost:  %.10f"% sess1_results[0],end='\r')
        
        MSE = MSE + sess1_results[0]
        len_train= len_train+1

    # Saving the most optimal model weights
    MSE_matrix[iter,0]= MSE/len_train
    if MSE_matrix[iter,0]< Max_error:
      Max_error = MSE_matrix[iter,0]
      saver_slave.save(sess1, save_path=save_path_slave)

# Master network architecture

In [0]:
o1 = tf.placeholder(shape=[None,480,640,8*factor],dtype=tf.float32)
o2 = tf.placeholder(shape=[None,480//2,640//2,16*factor],dtype=tf.float32)
o3 = tf.placeholder(shape=[None,480//4,640//4,32*factor],dtype=tf.float32)

x = tf.placeholder(shape=[None,480,640,3],dtype=tf.float32)
y = tf.placeholder(shape=[None,480,640,1],dtype=tf.float32)
conv_11 = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(concatenate([o1,x],axis =-1))
conv_12 = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_11)
conv_13 = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_12)
# pool_1  = MaxPooling2D(pool_size=(2,2))(concatenate([o1,conv_13],axis =-1))
pool_1  = MaxPooling2D(pool_size=(2,2))(conv_13)

conv_21 = Conv2D(16*factor, (3, 3), activation='relu', padding='same',trainable = True)(concatenate([o2,pool_1],axis =-1))
conv_22 = Conv2D(16*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_21)
# pool_2  = MaxPooling2D(pool_size=(2,2))(concatenate([o2,conv_22],axis =-1))
pool_2  = MaxPooling2D(pool_size=(2,2))(conv_22)

conv_31 = Conv2D(32*factor, (3, 3), activation='relu', padding='same',trainable = True)(concatenate([o3,pool_2],axis =-1))
conv_32 = Conv2D(32*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_31)
# pool_3  = MaxPooling2D(pool_size=(2,2))(concatenate([o3,conv_32],axis =-1))
pool_3  = MaxPooling2D(pool_size=(2,2))(conv_32)

conv_41 = Conv2D(64*factor, (3, 3), activation='relu', padding='same',trainable = True)(pool_3)
conv_42 = Conv2D(64*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_41)



up_5   = UpSampling2D(size = (2,2))(conv_42)
conv_51 = Conv2D(32*factor, (3, 3), activation='relu', padding='same',trainable = True)(up_5)
# conv_51 = Conv2D(32*factor, (3, 3), activation='relu', padding='same',trainable = True)(concatenate([up_5, conv_32], axis=-1))
conv_52 = Conv2D(32*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_51)
# out_3                  = conv_52


up_6   = UpSampling2D(size = (2,2))(conv_52)
conv_61 = Conv2D(16*factor, (3, 3), activation='relu', padding='same',trainable = True)(up_6)
# conv_61 = Conv2D(16*factor, (3, 3), activation='relu', padding='same',trainable = True)(concatenate([up_6, conv_22], axis=-1))
conv_62 = Conv2D(16*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_61)
# out_2                  = conv_62


up_7   = UpSampling2D(size = (2,2))(conv_62)
conv_71 = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(up_7)
# conv_71 = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(concatenate([up_7, conv_13], axis=-1))
conv_72 = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_71)
conv_73 = Conv2D(8*factor, (3, 3), activation='relu', padding='same',trainable = True)(conv_72)
# out_1                  = conv_73 

layer_final = Conv2D(1, (1, 1), activation='linear')(conv_73)

cost = tf.losses.mean_squared_error(y,layer_final)
auto_train = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)


# For save the model weight in each layer in the master network
saver_master=tf.train.Saver()
save_dir_master = 'Save_weighted/Master_network/'
if not os.path.exists(save_dir_master):
    os.makedirs(save_dir_master)
save_path_master = os.path.join(save_dir_master, 'best_validation')

# Training Master network

In [0]:
sess2 = tf.Session()
sess2.run(tf.global_variables_initializer())
################################################################################
Max_error =99999
MSE_matrix = np.zeros([num_epoch,2])
for iter in range(num_epoch):
    MSE =0
    len_train =0
    for current_batch_index in range(0,len(train_images),batch_size):
        current_batch = train_images[current_batch_index:current_batch_index+batch_size,:,:,:]
        current_label = train_labels[current_batch_index:current_batch_index+batch_size,:,:,:]
#         Train_start = time.clock()
        sess2_results = sess2.run([cost,auto_train,layer_final],
            feed_dict = {x:current_batch,
                        y:current_label,
                        o1:P_1,
                        o2:P_2,
                        o3:P_3
                        })
        
        print(' Actual model Iter: ', iter, " Cost:  %.10f"% sess2_results[0],end='\r')
        MSE = MSE + sess2_results[0]
        len_train= len_train+1
    MSE_matrix[iter,1]= MSE/len_train
    # Saving the most optimal model weights
    MSE_matrix[iter,1]= MSE/len_train
    if MSE_matrix[iter,1]< Max_error:
      Max_error = MSE_matrix[iter,1]
      saver_master.save(sess2, save_path=save_path_master)